In [1]:
import numpy as np
import pandas as pd
from surprise import KNNBaseline,Reader,Dataset,accuracy,KNNBasic
from surprise.model_selection import cross_validate,train_test_split

In [17]:
data = pd.read_excel('jester-data-3.xls',header=None)
data = data.rename({0:'Rate_count'},axis=1)
data['userId']=data.index+1


In [18]:
data

,Rate_count,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,userId
0,26,99.0,99.00,99.0,99.0,-1.65,99.00,-0.78,6.89,99.0,...,99.0,99.00,99.0,99.00,99.0,99.0,99.0,99.00,99.0,1
1,33,99.0,99.00,99.0,99.0,-9.27,99.00,-9.17,-8.59,99.0,...,99.0,-2.77,99.0,99.00,99.0,99.0,99.0,99.00,99.0,2
2,16,99.0,99.00,99.0,99.0,-6.12,99.00,-7.48,-7.77,99.0,...,99.0,99.00,99.0,99.00,99.0,99.0,99.0,99.00,99.0,3
3,24,99.0,0.05,99.0,99.0,-2.82,99.00,-4.85,-0.87,99.0,...,99.0,99.00,99.0,1.84,99.0,99.0,99.0,-4.08,99.0,4
4,22,99.0,99.00,99.0,99.0,-4.95,99.00,6.21,2.72,99.0,...,99.0,99.00,99.0,99.00,99.0,99.0,99.0,99.00,99.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24933,21,99.0,99.00,99.0,99.0,5.78,8.45,99.00,8.20,99.0,...,99.0,99.00,99.0,99.00,99.0,99.0,99.0,99.00,99.0,24934
24934,16,99.0,99.00,99.0,99.0,5.24,99.00,99.00,5.68,99.0,...,99.0,99.00,99.0,99.00,99.0,99.0,99.0,99.00,99.0,24935
24935,35,99.0,99.00,99.0,99.0,-3.93,99.00,-9.03,-2.67,99.0,...,99.0,99.00,99.0,99.00,99.0,99.0,99.0,99.00,99.0,24936
24936,21,99.0,99.00,99.0,99.0,9.42,9.66,99.00,9.17,99.0,...,99.0,99.00,99.0,99.00,99.0,99.0,99.0,99.00,99.0,24937


In [19]:
df =pd.melt(data.loc[:,1:] ,id_vars ='userId'  , value_name="rating")
df = df.rename({'variable':'JokesId'},axis=1)

In [20]:
df

,userId,JokesId,rating
0,1,1,99.0
1,2,1,99.0
2,3,1,99.0
3,4,1,99.0
4,5,1,99.0
...,...,...,...
2493795,24934,100,99.0
2493796,24935,100,99.0
2493797,24936,100,99.0
2493798,24937,100,99.0


In [21]:
df['rating'].replace(99.0,0,inplace=True)


In [22]:
df

,userId,JokesId,rating
0,1,1,0.0
1,2,1,0.0
2,3,1,0.0
3,4,1,0.0
4,5,1,0.0
...,...,...,...
2493795,24934,100,0.0
2493796,24935,100,0.0
2493797,24936,100,0.0
2493798,24937,100,0.0


In [23]:
reader = Reader(rating_scale=(-10,10))
dd = Dataset.load_from_df(df,reader)

In [24]:
trainset1 , testset1 = train_test_split(dd,random_state=7,test_size=0.20)

In [25]:
print("General information on the training sets we will be using \n")
print("1) Number of items in  dataset", trainset1.n_items)
print("2) Number of users in  dataset", trainset1.n_users)
print("3) Number of ratings in  dataset", trainset1.n_ratings)
print("4) Mean rating", trainset1.global_mean)

General information on the training sets we will be using 

1) Number of items in each dataset 100
2) Number of users in each dataset 24938
3) Number of ratings in each dataset 1995040
4) Mean rating 0.07355723694763013


In [26]:
sim_options = {'name': 'pearson_baseline', 'user_based': False}

In [27]:
# model = KNNBaseline(sim_options=sim_options)
model = KNNBasic(k=40,min_k=10,sim_options=sim_options)

In [28]:
model.fit(trainset1)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [29]:
pred = model.test(testset1)

In [30]:
accuracy.rmse(pred)

RMSE: 2.4271


2.427060633862265

In [33]:
predictions = pd.DataFrame(pred)
predictions=predictions.iloc[:,:-1]
# predictions['error'] = 
predictions

,uid,iid,r_ui,est
0,3739,84,0.00,0.006485
1,11951,48,-3.54,0.704056
2,497,93,2.14,0.000000
3,10212,22,0.00,-0.029480
4,1398,83,0.00,-0.016039
...,...,...,...,...
498755,14995,71,0.00,-0.512718
498756,3218,69,2.38,2.040746
498757,14459,78,0.00,-0.059453
498758,7010,88,-3.93,0.000000


In [65]:
from collections import defaultdict

In [59]:
def precision_k(predictions,k=10,threshold=0.0):
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))
    # print(user_est_true)
    precisions = dict()
    recalls=dict()
    for uid , user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        # print(urerRating[1])
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])


        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])
        precisions[uid] =  n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
    return precisions,recalls


In [62]:
precisions,recalls=precision_k(pred)


In [64]:
p=(sum(prec for prec in precisions.values()) / len(precisions))
r=(sum(rec for rec in recalls.values()) / len(recalls))
print(f'precision {p} recall {r}  \n\n')

precision 0.8059745152104062 recall 0.3692709794690942  


